# Criptosistema NTRU i atacs diversos

En aquest notebook es presenta la implementació del criptosistema NTRU i de quatre dels seus atacs: a la vulnerabilitat dels paràmetres, per força bruta, per trobada a mig camí i al KRP mitjançant reticles. Més enllà del codi, s'hi mostra l'execució tant del criptosistema com dels quatre atacs a mode d'exemple. En cas de voler realitzar altres execucions de manera més fluida es recomana fer ús del script present a aquest mateix repositori de Github.

## Implementació del criptosistema NTRU

En aquesta primera secció es llisten el seguit de funcions auxiliars necessàries pel correcte funcionament tant del criptosistema NTRU com dels diversos atacs, així com també la pròpia implementació del criptosistema NTRU.

### Funcions auxiliars

A grans trets, les funcions auxiliars que s'usen de manera recurrent en les diverses classes implementadesi que donen resposta a aspectes com ara la generació de polinomis ternaris, el càlcul d'inversos en anells polinomials o la lectura i escriptura de fitxers.

In [1]:
import logging
import random
import numpy as np
import itertools
import time
import base64
import sympy

In [2]:
def center_lift(pol): #Centra els coeficients del polinomi entrat per parametre a [-q/2, q/2].

    return pol.lift().map_coefficients(lambda c: c.lift_centered(), ZZ) 



def gen_ternary_pol(isf, N, d): #Es genera un polinomi ternari amb d (o d+1) coeficients iguals a 1 i d coeficients iguals a -1.

    key = [0]*N #Inicialitzem tots els N coeficients del polinomi a 0.
    index_list = [i for i in range(0, N)] #Prenem un llistat de tots els coeficients que valen 0.

    for j in range(d): #Per cadascuna de les d iteracions escollim dos coeficients buits i els omplim amb 1 i -1.
        index_1 = sage.misc.prandom.choice(index_list)
        index_list.remove(index_1)
        index_2 = sage.misc.prandom.choice(index_list)
        index_list.remove(index_2)
        key[index_1] = 1
        key[index_2] = -1

    if isf: #En cas que es tracti del polinomi f, s'afegeix un altre coeficient igual a 1.
        index_1 = sage.misc.prandom.choice(index_list)
        index_list.remove(index_1)

        key[index_1] = 1

    return key



def inv_pol_mod_prime(pol): #Donat un polinomi pertanyent a un anell quocient amb coeficients a un cos, retorna l'invers del polinomi a aquest anell prenent els coeficients a (-p/2,p/2). El polinomi retornat viu a l'anell dels polinomis amb coeficients als enters.
    
    try:
        return center_lift(pol^(-1))

    except ZeroDivisionError:
        raise


        
#Metode iteratiu de Newton
def inv_pol_mod_power_2(pol, aux, N, q): #Donat un polinomi pertanyent a un anell quocient amb coeficients als enters modul q una potencia de 2, retorna l'invers del polinomi a aquest anell prenent els coeficients a (-q/2,q/2). El polinomi retornat viu a l'anell dels polinomis am coeficients als enters.
    
    try:

        index = 2
        inv_pol = inv_pol_mod_prime(aux) #Es calcula l'invers modul 2 del polinomi original.
        pol2 = center_lift(pol) #Es centre lifta el polinomi original per tenir els coeficients a (-q/2,q/2).

        while index < q: #S'itera per cada potència de 2 fins arribar a q aplicant el mètode iteratiu de Newton.          
            index = index**2
            inv_pol = (2*inv_pol - pol2*(inv_pol)^(2))        
            aux_ring.<x> = PolynomialRing(ZZ.quotient(index))
            aux_ring = aux_ring.quotient(x^N - 1)
            inv_pol = center_lift(aux_ring(inv_pol)) #Es centre lifta el polinomi resultant per tenir els coeficients a (-q/2,q/2).
        
        return inv_pol
    
    except ZeroDivisionError:
        
        raise
        
        
        
def add_zeros(list_of_lists, N): #Afegeix 0 als darrers coeficients del polinomi que valen 0, ja que SageMath no els comptabilitza, per tal que totes les llistes tinguin mida N.

    for l in list_of_lists: #Per cadascun dels llistats de coeficients afegim els 0 al final que calgui.
        if (len(l)) < N: #Si el tamany de la llista es menor que N, cal afegir els 0 restants al final.
            zeros = [0]*(N - len(l))
            l.extend(zeros)



def split_message(message, list_of_messages, N): #Es divideix el missatge en missatges per poder ser encriptats.

    if len(message) <= N: #En cas que la mida del missatge sigui menor que el maxim establert, es pot encriptar.
        list_of_messages.append(message)

    else: #En cas contrari, cal separar-lo en submissatges de mida N.
        list_of_messages.append(message[:N])
        split_message(message[N:], list_of_messages, N)

        

def store_param_and_pub_key(filename, N, p, q, d, h): #Emmagatzema els parametres i la clau publica al fitxer indicat.

    h_coef_list = [] #Llista que conte els coeficients de la clau publica h.
    param_list = []  #Llista que conte els parametres publics en format string.
    h_coef_list = list(h) #S'obte la llista de coeficients de la clau publica h.

    if len(h_coef_list) < N: #En cas que la llista de coeficients sigui menor que N, cal afegir-hi 0 al final.
        zeros = [0]*(N - len(h_coef_list))
        h_coef_list.extend(zeros)

    string_list = [str(c) for c in h_coef_list] #Es passa dels coeficients enters a els coeficients en string.
    h_c_string = ",".join(string_list) #S'uneixen els coeficients en un sol string separat per comes.
    param_list.append(str(N)) #S'afegeixen els parametres publics en format string.
    param_list.append(str(p))
    param_list.append(str(q))
    param_list.append(str(d))
    param_list.append(h_c_string)
    param_text = ";".join(param_list) #S'uneixen els parametres publics en un sol string separat pel caracter ";".

    f = open(filename, "w") #S'escriu el text en el fitxer corresponent.
    f.write(param_text)
    f.close()

    

def store_param_and_priv_keys(filename, N, p, q, d, f, g): #Emmagatzema els parametres i les claus privades al fitxer indicat.

    f_coef_list = [] #Llista que conte els coeficients de la clau privada f.
    g_coef_list = [] #Llista que conte els coeficients de la clau privada g.
    param_list = []  #Llista que conte els parametres publics en format string.
    f_coef_list = list(f) #S'obte la llista de coeficients de la clau privada f.
    g_coef_list = list(g) #S'obte la llista de coeficients de la clau privada g.

    if len(f_coef_list) < N: #En cas que la llista de coeficients sigui menor que N, cal afegir-hi 0 al final.
        zeros = [0]*(N - len(f_coef_list))
        f_coef_list.extend(zeros)

    if len(g_coef_list) < N: #En cas que la llista de coeficients sigui menor que N, cal afegir-hi 0 al final.
        zeros = [0]*(N - len(g_coef_list))
        g_coef_list.extend(zeros)

    string_list = [str(c) for c in f_coef_list] #Es passa dels coeficients enters a els coeficients en string.
    f_c_string = ",".join(string_list) #S'uneixen els coeficients en un sol string separat per comes.

    string_list = [str(c) for c in g_coef_list] #Es passa dels coeficients enters a els coeficients en string.
    g_c_string = ",".join(string_list) #S'uneixen els coeficients en un sol string separat per comes.
    param_list.append(str(N)) #S'afegeixen els parametres publics en format string.
    param_list.append(str(p))
    param_list.append(str(q))
    param_list.append(str(d))
    param_list.append(f_c_string)
    param_list.append(g_c_string)
    param_text = ";".join(param_list) #S'uneixen els parametres publics en un sol string separat pel caràcter ";".

    f = open(filename, "w") #S'escriu el text en el fitxer corresponent.
    f.write(param_text)
    f.close()



def read_encrypted_file(filename, N): #Llegeix el fixer encriptat i en converteix el text a una llista que conte subllistes referents als polinomis corresponents als missatges encriptats per cada linia de text.

    f = open(filename, "r") #Es llegeix el fitxer que conte el missatge encriptat.
    coef = f.read()
    f.close()

    lines_list  = [] #Llista que conte el text corresponent a cadascuna de les linies encriptades.
    pol_lines_list  = [] #llista que conte subllistes referents als coeficients dels missatges encriptats per cada linia de text.
    int_coef_line_list = [] #Llista que conte els coeficients (enters) dels polinomis corresponents a una linia del text encriptat.
    pol_line_list  = [] #Llista que conte els polinomis corresponents al text encriptat d'una linia de text.
    lines_list  = coef.split(";") #Es separen els coeficients llegits del fitxer per línies de text.
    lines_list.pop() #S'elimina l'ultim element ja que es correspon amb una linia buida.
    z_pol_ring.<x> = PolynomialRing(ZZ) #Es crea l'anell de polinomis amb coeficients enters.

    for line in lines_list: #Per cada linia de text es transformen els coeficients en els polinomis corresponents.
        int_coef_line_list = [int(c) for c in line.split(',')] #Es passen de coeficients de tipus string a tipus int.
        i=0 #S'inicia el comptador a 0.

        for coef in int_coef_line_list: #S'itera pels coeficients de cada linia de text.
            if i%(N) == 0: #Cada N coeficiens cal crear un nou polinomi i afegir-lo a la llista.
                pol_line_list.append(z_pol_ring(int_coef_line_list[i:i+N]))
            
            i+=1    

        pol_lines_list.append(pol_line_list) #S'afegeixen els polinomis corresponents a una linia de text.
        pol_line_list = [] #Es buida la llista corresponent als polinomis d'una linia de text.

    return pol_lines_list



def pol_to_coef(pol_list, N): #Es converteixen els polinomis en llistes que contenen els seus coeficients.

    coef_list = [] #Llista de subllistes que conte els coeficients ordenats dels polinomis.

    for pol in pol_list: #Es trasforma cada polinomi a una llista dels seus coeficients.
        c_l = list(pol)

        if len(c_l) < N: #Si la llista dels seus coeficients es menor que N, s'afegeixen els 0 que calgui al final.  
            zeros = [0]*(N - len(c_l))
            c_l.extend(zeros)

        coef_list.extend(c_l)


    return coef_list



def is_ternary(pol, N, d): #Metode que indica si un polinomi compleix les condicions propies de la clau privada g.

    ternary = True
    index = 0
    num_ones = 0
    num_neg_ones = 0
    num_zeros = 0

    while ternary and index < len(pol): #Es recorren els coeficients del polinomi fins a descartar-lo o arribar al final.

        if pol[index] == 1: #En cas que el valor del coeficient sigui 1, n'augmentem el comptador.
            num_ones +=1

            if num_ones > d: #Es comprova si el nombre de coeficients amb valor 1 ha superat el requerit.
                ternary = False

        elif pol[index] == -1: #En cas que el valor del coeficient sigui -1, n'augmentem el comptador.
            num_neg_ones +=1

            if num_neg_ones > d: #Es comprova si el nombre de coeficients amb valor -1 ha superat el requerit.
                ternary = False

        elif pol[index] == 0: #En cas que el valor del coeficient sigui 0, n'augmentem el comptador.
            num_zeros +=1

            if num_zeros > N-d-d: #Es comprova si el nombre de coeficients amb valor 0 ha superat el requerit.
                ternary = False
        else: #En cas contrari, el polinomi no es de la forma requerida per la clau privada g.
            ternary = False

        index+=1

    return ternary



def gen_message(N, p): #Genera un missatge aleatori dins dels limits establerts pels parametres publics.
    
    message = []
    
    a = floor((p-1)/2) #Rang de valors que poden prendre els coeficients del missatge.
    
    for i in range(0, N): #S'itera per cadascun dels coeficients del missatge.
        message.append(random.randint(-a,a))
    
    return message

### Criptosistema NTRU

La classe NTRU es correspon amb la implementació del criptosistema NTRU. Permet obtenir les claus públiques i privades, encriptar i desencriptar missatges a partir dels paràmetres $N$, $p$, $q$ i $d$ entrats per teclat. A més a més, tant les claus com els textos encriptats i desencriptats es poden llegir i escriure en fitxers de text.

In [3]:
class Ntru:
    
    N = None
    p = None
    q = None
    d = None
    f = None
    g = None
    h = None
    r = None
    f_inv_mod_p = None
    f_inv_mod_q = None
    r_ring = None
    r_ring_mod_p = None
    r_ring_mod_q = None
    r_ring_mod_2 = None
    
    
    
    def __init__(self, N, p, q, d): #Inicialització dels parametres i dels anells que es requereixen per fer funcionar el criptosistema NTRU.
        
        try:
            assert (N - 1)/2 >= d #Comprovem la condició 1 dels parametres.
            
        except AssertionError: 
            print('Cal que (N - 1)/2 >= d')
        
        try:  
            assert q > (6*d + 1)*p #Comprovem la condició 2 dels parametres.
        
        except AssertionError:
            print('Cal que q > (6*d + 1)*p')

        try:
            assert q.is_power_of(2) #Comprovem que q sigui potència de 2.
        
        except AssertionError:
            print('q ha de ser potència de 2.')
        
        self.N = N
        self.p = p
        self.q = q
        self.d = d
        
        aux_ring.<x> = PolynomialRing(ZZ)
        self.r_ring = aux_ring.quotient(x^self.N - 1)
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(self.p)) 
        self.r_ring_mod_p = aux_ring.quotient(x^self.N - 1) 
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(self.q)) 
        self.r_ring_mod_q = aux_ring.quotient(x^self.N - 1) 
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(2)) 
        self.r_ring_mod_2 = aux_ring.quotient(x^self.N - 1)
        
        print("Classe NTRU instanciada amb els paràmetres N={}, p={}, q={}, d={}".format(self.N, self.p, self.q, self.d))
        
    

    def gen_priv_keys(self): #Es calculen les claus privades f i g, aixi com tambe els polinomis f_inv_mod_p i f_inv_mod_q.
        
        trobat = False
        g_list = gen_ternary_pol(False, self.N, self.d) #S'obtenen els coeficients del polinomi g.
        self.g = self.r_ring(g_list)
        
        while not trobat: #Mentre no s'hagi trobat un f que disposi d'inversos modul p i q, se'n busquen de nous.
            f_list = gen_ternary_pol(True, self.N, self.d) #S'obtenen els coeficients del polinomi g.
            self.f = self.r_ring(f_list)
            
            if not self.p.is_power_of(2): #Cas en que p no es potencia de 2.
                aux_f_p = self.r_ring_mod_p(f_list)

                if aux_f_p.is_unit(): #En cas que f tingui invers modul p, es calcula.
                    self.f_inv_mod_p = inv_pol_mod_prime(aux_f_p)
                    aux_f_q = self.r_ring_mod_2(f_list)

                    if aux_f_q.is_unit(): #En cas que f tingui invers modul q, s'aplica l'algorisme iteratiu de Newton per obtenir l'invers modul q de f.             
                        self.f_inv_mod_q = inv_pol_mod_power_2(self.r_ring_mod_q(f_list), aux_f_q, self.N, self.q)
                        trobat = True #S'indica que s'ha trobat una clau privada f valida i els seus inversos moduls p i q.
                        
            else: #Cas en que p es potencia de 2.
                aux_f_p = self.r_ring_mod_2(f_list)

                if aux_f_p.is_unit(): #En cas que f tingui invers modul p, es calcula.
                    self.f_inv_mod_p = inv_pol_mod_power_2(self.r_ring_mod_p(f_list), aux_f_p, self.N, self.p)
                    aux_f_q = self.r_ring_mod_2(f_list)

                    if aux_f_q.is_unit(): #En cas que f tingui invers modul q, s'aplica l'algorisme iteratiu de Newton per obtenir l'invers modul q de f.             
                        self.f_inv_mod_q = inv_pol_mod_power_2(self.r_ring_mod_q(f_list), aux_f_q, self.N, self.q)
                        trobat = True #S'indica que s'ha trobat una clau priva f valida i els seus inversos moduls p i q.      
        
   

    def gen_pub_key(self): #Es calcula la clau publica h.
        
        z_pol_ring.<x> = PolynomialRing(ZZ)
        h_z = z_pol_ring(list(self.f_inv_mod_q))*z_pol_ring(list(self.g)) #El producte entre f i g es fa a l'anell dels polinomis amb coeficients enters.        
        self.h = center_lift(self.r_ring_mod_q(h_z)) #Es prenen moduls q i (x^N-1) i es centre liften els coeficients. 

        
        
    def gen_keys(self, pub_filename, priv_filename): #Es calculen tant les claus privades com la publica.
        
        self.gen_priv_keys()
        self.gen_pub_key()
        store_param_and_pub_key(pub_filename, self.N, self.p, self.q, self.d, self.h)
        store_param_and_priv_keys(priv_filename, self.N, self.p, self.q, self.d, self.f, self.g)
        

    
    def set_param_and_pub_key(self, filename): #Estableix els parametres i la clau publica segons els valors del fitxer indicat.
        
        f = open(filename, "r") #Es llegeix el fitxer que conte els parametres i la clau publica.
        params = f.read()
        f.close()
        
        param_list = [] #Llista que conte els parametres publics en format string.
        h_int_list = [] #Llista que conte els coeficients de la clau publica h en format int.
        param_list  = params.split(";") #Es separen els parametres publics llegits del fitxer de text.    
        param_list[-1] = param_list[-1].split(",") #Es separen els coeficients de la clau publica h llegida del fitxer.
        h_int_list = [int(c) for c in param_list[-1]] #Es passen de coeficients de tipus string a tipus int.
        
        z_pol_ring.<x> = PolynomialRing(ZZ)
        self.N = Integer(param_list[0]) #S'estableixen els nous valors dels parametres publics.
        self.p = Integer(param_list[1])
        self.q = Integer(param_list[2])
        self.d = Integer(param_list[3])
        
        aux_ring.<x> = PolynomialRing(ZZ)
        self.r_ring = aux_ring.quotient(x^self.N - 1)
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(self.p))
        self.r_ring_mod_p = aux_ring.quotient(x^self.N - 1)
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(self.q)) 
        self.r_ring_mod_q = aux_ring.quotient(x^self.N - 1)
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(2)) 
        self.r_ring_mod_2 = aux_ring.quotient(x^self.N - 1)
        self.h = z_pol_ring(h_int_list)
        
        
        
    def set_param_and_priv_keys(self, filename): #Estableix els parametres i les claus privades segons els valors del fitxer indicat.
        
        f = open(filename, "r") #Es llegeix el fitxer que conte els parametres i la clau publica.
        params = f.read()
        f.close()
        
        param_list = [] #Llista que conte els parametres publics en format string.
        f_int_list = [] #Llista que conte els coeficients de la clau privada f en format int.
        g_int_list = [] #Llista que conte els coeficients de la clau privada g en format int.
        param_list  = params.split(";") #Es separen els parametres publics llegits del fitxer de text.
        param_list[-2] = param_list[-2].split(",") #Es separen els coeficients de la clau privada f llegida del fitxer.        
        param_list[-1] = param_list[-1].split(",") #Es separen els coeficients de la clau privada g llegida del fitxer.
        f_int_list = [int(c) for c in param_list[-2]] #Es passen de coeficients de tipus string a tipus int.
        g_int_list = [int(c) for c in param_list[-1]] #Es passen de coeficients de tipus string a tipus int.
       
        z_pol_ring.<x> = PolynomialRing(ZZ)
        self.N = Integer(param_list[0]) #S'estableixen els nous valors dels parametres publics.
        self.p = Integer(param_list[1])
        self.q = Integer(param_list[2])
        self.d = Integer(param_list[3])
        aux_ring.<x> = PolynomialRing(ZZ)
        self.r_ring = aux_ring.quotient(x^self.N - 1)
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(self.p))
        self.r_ring_mod_p = aux_ring.quotient(x^self.N - 1)
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(self.q)) 
        self.r_ring_mod_q = aux_ring.quotient(x^self.N - 1)
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(2)) 
        self.r_ring_mod_2 = aux_ring.quotient(x^self.N - 1)
        
        self.f = self.r_ring(f_int_list)
        self.g = self.r_ring(g_int_list)
        self.f_inv_mod_q = inv_pol_mod_power_2(self.r_ring_mod_q(f_int_list), self.r_ring_mod_2(f_int_list), self.N, self.q)
        
        if not self.p.is_power_of(2):
            self.f_inv_mod_p = inv_pol_mod_prime(self.r_ring_mod_p(f_int_list))
            
        else:
            self.f_inv_mod_p = inv_pol_mod_power_2(self.r_ring_mod_p(f_int_list), self.r_ring_mod_2(f_int_list), self.N, self.p)
            
            
     
    def encryption(self, m): #S'encripta el missatge entrat per parametre.
        
        try:
            assert len(m) <= self.N #Es comprova que el missatge tingui el tamany permes.
            
        except AssertionError: 
            print('Missatge massa llarg')
        
        z_pol_ring.<x> = PolynomialRing(ZZ)
        m_pol = z_pol_ring(m)
        self.r = z_pol_ring(gen_ternary_pol(False, self.N, self.d)) #Es calcula el polinomi aleatori r per aquest missatge en concret.
        e = self.p*self.h*self.r + m_pol
        e = center_lift(self.r_ring_mod_q(e))
        
        return e
        
        
        
    def decryption(self, e): #Es desencripta el missatge entrat per parametre.
        
        z_pol_ring.<x> = PolynomialRing(ZZ)
        a = z_pol_ring(list(self.f))*e #Es calcula la primera de les dues convolucions.
        a = center_lift(self.r_ring_mod_q(a)) #Es prenen moduls q i (x^N-1) i es centre liften els coeficients.
        b = z_pol_ring(list(a))*self.f_inv_mod_p #Es calcula la segona de les dues convolucions.
        b = center_lift(self.r_ring_mod_p(b)) #Es prenen moduls p i (x^N-1) i es centre liften els coeficients.

        return b
        
        
        
    def encrypt_message(self, list_of_messages): #S'encripten el conjunt de submissatges.
        
        encrypted_list = []
        
        for i in range(len(list_of_messages)): #Per cadascun dels submissatges es crida a la funcio que els encripta.
            encrypted_list.append(self.encryption(list(list_of_messages[i])))
        
        return encrypted_list #Es retorna una llista amb tots els submissatges encriptats.
    
    
    
    def decrypt_message(self, list_of_encrypted): #Es desencripten el conjunt de submissatges encriptats.
        
        decrypted_list = []
        
        for i in range(len(list(list_of_encrypted))): #Per cadascun dels submissatges es crida a la funcio que els desencripta.
            decrypted_list.append(list(self.decryption(list_of_encrypted[i])))
        
        return decrypted_list #Es retorna una llista amb tots els submissatges desencriptats.
    
    
    
    def encrypt_file(self, m_filename, e_filename): #Donats dos fitxers, llegeix el text del primer i n'escriu els coeficients dels polinomis corresponents a la seva encriptacio al segon fitxer.
        
        write_lines_list = [] #Llista on cada element es correspon amb el text encriptat d'una linia.
        coef_list = [] #Llista on els elements son els coeficients (enters) dels polinomis corresponents a l'encriptacio d'una lina de text.
        string_list = [] #Llista on els elements son els coeficients (strings) dels polinomis corresponents a l'encriptacio d'una lina de text.
        lines_list = [] #Llista on els elements son cadascuna de les linies de text del fitxer a encriptar.
        m_list = [] #llista que conte els submissatges per cada linia de text a encriptar
        
        f = open(m_filename, "rb") #Es llegeix el fitxer de text a encriptar.
        lines_list = f.read().splitlines()
        f.close()
        
        for line in lines_list: #S'encripten les linies del fitxer una a una.
            m_bits = np.unpackbits(np.frombuffer(line, dtype=np.uint8)) #Es passa el text al seu equivalent en bits.
            m_bits = np.trim_zeros(m_bits,'b') #S'eliminen els 0 sobrants del final.
            split_message(list(m_bits), m_list, self.N) #Es divideix cada linia en submissatges per poder ser encriptats.
            e_list = self.encrypt_message(m_list) #S'encripten els submissatges.
            m_list = [] #Es torna a buidar la llista de submissatges.
            coef_list = pol_to_coef(e_list, self.N) #Es passa dels polinomis a la llista dels seus coeficients.
            string_list = [str(c) for c in coef_list] #Es passa dels coeficients enters a els coeficients en string.
            c_string = ",".join(string_list) #S'uneixen els coeficients en un sol string separat per comes.
            c_string +=";" #Per indicar el fi de la linia s'afegeix un caracter ;.
            write_lines_list.append(c_string) #S'afegeix el string resultant a la llista de linies encriptades.
        
        f = open(e_filename, "w") #S'escriuen les linies encriptades al fitxer que ha de contenir el missatge encriptat.
        f.writelines(write_lines_list)
        f.close()

    
    
    def decrypt_file(self, e_filename, d_filename): #Donats dos fitxers, llegeix els coeficients dels polinomis corresponents al text encriptat i escriu el missatge desencriptat al segon fitxer.
                
        decrypted_lines_list = [] #Llista on els elements son les linies de text desencriptades.
        b_list = [] #Llista que conte subllistes corresponents als bits dels submissatges d'una linia de text desencriptada.
        b_def_list = [] #Llista que conte els bits dels submissatges unificats d'una linia de text desencriptada.
        pol_lines_list = [] #Llista que conte subllistes referents als coeficients dels missatges encriptats per cada linia de text.
        pol_lines_list = read_encrypted_file(e_filename, self.N) #Es llegeix el text encriptat del fitxer corresponent.

        for line in pol_lines_list: #Es desencripten les linies del fitxer una a una.
            b_list = self.decrypt_message(line) #Es desencripta una linia.
            add_zeros(b_list, self.N) #S'afegeixen els 0 que calgui al final de cada polinomi.
            b_def_list = [bit for polynomial in b_list for bit in polynomial] #S'unifiquen els coeficients dels polinomis a una sola linia.
            dec_text = "".join(map(chr, np.packbits(np.array(list(b_def_list))))) #Es transformen els bits al text en string corresponent.
            decrypted_lines_list.append(dec_text+"\n") #S'afegeix un salt de linia al final de cada linia.
        
        f = open(d_filename, "w") #S'escriu el text desencriptat al fitxer corresponent.
        f.writelines(decrypted_lines_list)
        f.close()
        
        
        
    def decryption_no_file(self, e): #Desencripta missatges sense lectura de fitxers.
        
        b = self.decryption(e)
        b_list = list(b)
        
        if (len(b_list)) < self.N: #Si el tamany de la llista es menor que N, cal afegir els 0 restants al final.
            zeros = [0]*(self.N - len(b_list))
            b_list.extend(zeros)

        return b_list

    
    
    def get_pub_key(self): #Retorna la clau publica.
        
        return self.h
    
    
    
    def set_pub_key(self, new_h): #Assigna la clau publica el polinomi per paràmetre.
        
        self.h = new_h


Tot seguit es genera una instància del criptosistema NTRU, s'emmagatzemen les claus en fitxers de text i s'encripta i desenecripta un fitxer de text. A més a més, es computa el temps requerit per a desencriptar-lo.

In [4]:
a = Ntru(17, 8, 512, 2)
a.gen_keys("public_key.txt", "private_keys.txt")

Classe NTRU instanciada amb els paràmetres N=17, p=8, q=512, d=2


In [5]:
a.set_param_and_pub_key("public_key.txt")
a.encrypt_file("missatge.txt", "encriptacio_missatge.txt")
a.set_param_and_priv_keys("private_keys.txt")
start5_time = time.time()
a.decrypt_file("encriptacio_missatge.txt","desencriptacio_missatge.txt")
print("El procés de desencriptació del fitxer ha tardat %s segons." % (time.time() - start5_time))

El procés de desencriptació del fitxer ha tardat 0.058527469635009766 segons.


A continuació es genera una instància del criptosistema NTRU, i es duu a terme el procés de generació de claus, encriptació i desencriptació d'un missatge sense fer ús de fitxers de text. A més a més, es computa el temps requerit per a desencriptar-lo.

In [6]:
N, p, q, d = 17, 8, 512, 2
nfa = Ntru(N, p, q, d)
nfmessage = gen_message(N, p)
nfa.gen_priv_keys()
nfa.gen_pub_key()
nfh = nfa.get_pub_key()
nfenc = nfa.encryption(nfmessage)
nfstart1_time = time.time()
nfdec = nfa.decryption_no_file(nfenc)
print("Missatge que es vol ecriptar: {}".format(nfmessage))
print("Missatge encriptat: {}".format(list(nfenc)))
print("Missatge desencriptat: {}".format(nfdec))
print("El procés de desencriptació del missatge ha tardat %s segons." % (time.time() - nfstart1_time))

Classe NTRU instanciada amb els paràmetres N=17, p=8, q=512, d=2
Missatge que es vol ecriptar: [1, 1, 3, -1, -3, 2, 0, 3, -3, -1, 1, -3, 2, 0, 1, -3, -2]
Missatge encriptat: [137, -87, -77, -113, -83, 18, -160, 171, 253, 135, -127, 13, -214, -48, -151, -19, -162]
Missatge desencriptat: [1, 1, 3, -1, -3, 2, 0, 3, -3, -1, 1, -3, 2, 0, 1, -3, -2]
El procés de desencriptació del missatge ha tardat 0.0018358230590820312 segons.


## Implementació d'atacs al criptosistema NTRU

En aquesta secció es presenten la implementació del quatre atacs enunciats prèviament i un exemple d'execució de cadascun d'ells.

### Atac a la vulnerabilitat dels paràmetres

La classe Param_vulnerability es correspon amb la implementació de l'atac a la vulnerabilitat dels paràmetres. Permet desencriptar missatges a partir dels paràmetres públics sempre i quan $p$ i $q$ no siguin coprimers. A més a més, els textos encriptats i desencriptats es poden llegir i escriure en fitxers de text.

In [7]:
class Param_vulnerability:
    
    N = None
    p = None
    q = None
    d = None
    r = None
    r_ring = None
    r_ring_mod_p = None
    r_ring_mod_q = None
    r_ring_mod_2 = None

   

    def __init__(self, *args): #Inicialitzacio dels parametres i dels anells que es requereixen per fer funcionar l'atac al KRP basat en reticles sense llegir un fitxer.
        
        if len(args)==1:
            f = open(args[0], "r") #Es llegeix el fitxer que conte els parametres publics.
            params = f.read()
            f.close()

            param_list = [] #Llista que conte els parametres publics en format string.
            param_list  = params.split(";")
            param_list[-1] = param_list[-1].split(",") #Es separen els coeficients de la clau publica h llegida del fitxer.
            h_int_list = [int(c) for c in param_list[-1]] #Es passen de coeficients de tipus string a tipus int.

            z_pol_ring.<x> = PolynomialRing(ZZ)
            self.N = Integer(param_list[0]) #S'estableixen els nous valors dels parametres publics.
            self.p = Integer(param_list[1])
            self.q = Integer(param_list[2])
            self.d = Integer(param_list[3])
            self.h = z_pol_ring(h_int_list)
            
        else:
            try:
                assert (args[0] - 1)/2 >= args[3] #Comprovem la condició 1 dels parametres.

            except AssertionError: 
                print('Cal que (N - 1)/2 >= d')

            try:  
                assert args[2] > (6*args[3] + 1)*args[1] #Comprovem la condició 2 dels parametres.

            except AssertionError:
                print('Cal que q > (6*d + 1)*p')

            try:
                assert args[2].is_power_of(2) #Comprovem que q sigui potència de 2.

            except AssertionError:
                print('q ha de ser potència de 2.')

            self.N = args[0]
            self.p = args[1]
            self.q = args[2]
            self.d = args[3]
        
        aux_ring.<x> = PolynomialRing(ZZ)
        self.r_ring = aux_ring.quotient(x^self.N - 1)
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(self.p)) 
        self.r_ring_mod_p = aux_ring.quotient(x^self.N - 1) 
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(self.q)) 
        self.r_ring_mod_q = aux_ring.quotient(x^self.N - 1) 
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(2)) 
        self.r_ring_mod_2 = aux_ring.quotient(x^self.N - 1)
        
        
        
    def attack_file(self, e_filename, d_filename): #S'ataca el primer fitxer entrat per parametre i es guarda el missatge al segon fitxer entrat per parametre
        
        decrypted_lines_list = [] #Llista on els elements son les linies de text desencriptades.
        a_list = [] #Llista que conte subllistes corresponents als bits dels submissatges d'una linia de text desencriptada.
        a_def_list = [] #Llista que conte els bits dels submissatges unificats d'una linia de text desencriptada.
        pol_lines_list = [] #llista que conte subllistes referents als coeficients dels missatges encriptats per cada linia de text.
        gc = gcd(self.q,self.p)
        
        if gc == 1:
            print("{} i {} coprimers, atac impossible de realitzar.".format(self.p, self.q))
        
        else:  
            pol_lines_list = read_encrypted_file(e_filename, self.N) #Es llegeix el text encriptat del fitxer corresponent.
    
            for line in pol_lines_list: #Es desencripten les linies del fitxer una a una.
                a_list = self.attack_message(line, gc) #Es desencripta una linia.
                add_zeros(a_list, self.N) #S'afegeixen els 0 que calgui al final de cada polinomi.
                a_def_list = [bit for polynomial in a_list for bit in polynomial] #S'unifiquen els coeficients dels polinomis a una sola linia.
                dec_text = "".join(map(chr, np.packbits(np.array(list(a_def_list))))) #Es transformen els bits al text en string corresponent.
                decrypted_lines_list.append(dec_text+"\n") #S'afegeix un salt de linia al final de cada linia.
        
        f = open(d_filename, "w") #S'escriu el text desencriptat al fitxer corresponent.
        f.writelines(decrypted_lines_list)
        f.close()
        
        
    
    def attack_message(self, list_of_encrypted, gc): #S'ataquen el conjunt de submissatges encriptats.
        
        attacked_list = []
        
        for i in range(len(list(list_of_encrypted))): #Per cadascun dels submissatges es crida a la funcio que els ataca.
            attacked_list.append(list(self.attack(list_of_encrypted[i], gc)))
        
        return attacked_list #Es retorna una llista amb tots els submissatges atacats.
    
    
            
    def attack(self, e, gc): #S'ataca el missatge entrat per parametre.
        
        z_pol_ring.<x> = PolynomialRing(ZZ)
        aux_ring.<x> = PolynomialRing(ZZ.quotient(Integer(gc))) 
        r_ring_mod_c = aux_ring.quotient(x^self.N - 1) #Es pren modul c per desencriptar el missatge encriptat.
        b = center_lift(r_ring_mod_c(list(e)))
        
        return b
    
    
    
    def attack_no_file(self, e): #S'ataca el missatge entrat per parametre sense llegir de fitxers.
        
        gc = gcd(self.q, self.p)
        
        if gc == 1:
            print("{} i {} coprimers, atac impossible de realitzar.".format(self.p, self.q))
            b = [0]
            
        else:
            z_pol_ring.<x> = PolynomialRing(ZZ)
            aux_ring.<x> = PolynomialRing(ZZ.quotient(Integer(gc)))
            r_ring_mod_c = aux_ring.quotient(x^self.N - 1)
            b = list(center_lift(r_ring_mod_c(list(e))))
            
            if (len(b)) < self.N: #Si el tamany de la llista es menor que N, cal afegir els 0 restants al final.
                zeros = [0]*(self.N - len(b))
                b.extend(zeros)

        return b

Tot seguit s'ataca el fitxer encriptat per la instància del criptosistema NTRU anterior mitjançant l'atac a la vulnerabilitat dels paràmetres i s'emmagatzema el missatge resultant a un fitxer de de text. A més a més, es computa el temps requerit per a atacar-lo.

In [8]:
v = Param_vulnerability("public_key.txt")
start4_time = time.time()
v.attack_file("encriptacio_missatge.txt","atac_vp_missatge.txt")
print("L'atac a la vulerabilitat dels paràmetres ha tardat %s segons." % (time.time() - start4_time))

L'atac a la vulerabilitat dels paràmetres ha tardat 0.07418179512023926 segons.


A continuació s'ataca mitjançant l'atac a la vulnerabilitat dels paràmetres el missatge encriptat prèviament sense fer ús de fitxers de text. A més a més, es computa el temps requerit per a desencriptar-lo.

In [9]:
nfpa = Param_vulnerability(N, p, q, d)
nfstart3_time = time.time()
nfpv = nfpa.attack_no_file(nfenc)
print("Missatge que es vol ecriptar: {}".format(nfmessage))
print("Missatge encriptat: {}".format(list(nfenc)))
print("Missatge desencriptat: {}".format(nfpv))
print("L'atac a la vulerabilitat dels paràmetres ha tardat %s segons." % (time.time() - nfstart3_time))

Missatge que es vol ecriptar: [1, 1, 3, -1, -3, 2, 0, 3, -3, -1, 1, -3, 2, 0, 1, -3, -2]
Missatge encriptat: [137, -87, -77, -113, -83, 18, -160, 171, 253, 135, -127, 13, -214, -48, -151, -19, -162]
Missatge desencriptat: [1, 1, 3, -1, -3, 2, 0, 3, -3, -1, 1, -3, 2, 0, 1, -3, -2]
L'atac a la vulerabilitat dels paràmetres ha tardat 0.0012233257293701172 segons.


### Atac per força bruta

La classe Brute_force es correspon amb la implementació de l'atac per força bruta. Permet desencriptar missatges a partir dels paràmetres públics i de la clau pública. A més a més, els textos encriptats i desencriptats es poden llegir i escriure en fitxers de text.

In [10]:
class Brute_force:
    
    N = None
    p = None
    q = None
    d = None
    r = None
    h = None
    f = None
    f_inv_mod_p = None
    f_inv_mod_q = None
    g = None
    f_list = None
    g_list = None
    r_ring = None
    r_ring_mod_p = None
    r_ring_mod_q = None
    r_ring_mod_2 = None
    
      
        
    def __init__(self, *args): #Inicialitzacio dels parametres i dels anells que es requereixen per fer funcionar l'atac al KRP basat en reticles sense llegir un fitxer.
        
        if len(args)==1:
            f = open(args[0], "r") #Es llegeix el fitxer que conte els parametres publics.
            params = f.read()
            f.close()

            param_list = [] #Llista que conte els parametres publics en format string.
            param_list  = params.split(";")
            param_list[-1] = param_list[-1].split(",") #Es separen els coeficients de la clau publica h llegida del fitxer.
            h_int_list = [int(c) for c in param_list[-1]] #Es passen de coeficients de tipus string a tipus int.

            z_pol_ring.<x> = PolynomialRing(ZZ)
            self.N = Integer(param_list[0]) #S'estableixen els nous valors dels parametres publics.
            self.p = Integer(param_list[1])
            self.q = Integer(param_list[2])
            self.d = Integer(param_list[3])
            self.h = z_pol_ring(h_int_list)
            
        else:
            try:
                assert (args[0] - 1)/2 >= args[3] #Comprovem la condició 1 dels parametres.

            except AssertionError: 
                print('Cal que (N - 1)/2 >= d')

            try:  
                assert args[2] > (6*args[3] + 1)*args[1] #Comprovem la condició 2 dels parametres.

            except AssertionError:
                print('Cal que q > (6*d + 1)*p')

            try:
                assert args[2].is_power_of(2) #Comprovem que q sigui potència de 2.

            except AssertionError:
                print('q ha de ser potència de 2.')

            self.N = args[0]
            self.p = args[1]
            self.q = args[2]
            self.d = args[3]
        
        aux_ring.<x> = PolynomialRing(ZZ)
        self.r_ring = aux_ring.quotient(x^self.N - 1)
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(self.p)) 
        self.r_ring_mod_p = aux_ring.quotient(x^self.N - 1) 
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(self.q)) 
        self.r_ring_mod_q = aux_ring.quotient(x^self.N - 1) 
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(2)) 
        self.r_ring_mod_2 = aux_ring.quotient(x^self.N - 1)
        
        
        
    def decryption(self, e): #Es desencripta el missatge entrat per parametre.
        
        z_pol_ring.<x> = PolynomialRing(ZZ)
        a = z_pol_ring(list(self.f))*e #Es calcula la primera de les dues convolucions.
        a = center_lift(self.r_ring_mod_q(a)) #Es prenen moduls q i (x^N-1) i es centre liften els coeficients.
        b = z_pol_ring(list(a))*self.f_inv_mod_p #Es calcula la segona de les dues convolucions.
        b = center_lift(self.r_ring_mod_p(b)) #Es prenen moduls p i (x^N-1) i es centre liften els coeficients.

        return b
    
    
    
    def decryption_no_file(self, e): #Desencripta missatges sense lectura de fitxers.
        
        b = self.decryption(e)
        b_list = list(b)
        
        if (len(b_list)) < self.N: #Si el tamany de la llista es menor que N, cal afegir els 0 restants al final.
            zeros = [0]*(self.N - len(b_list))
            b_list.extend(zeros)

        return b_list
    
    
    
    def decrypt_message(self, list_of_encrypted): #Es desencripten el conjunt de submissatges encriptats.
        
        decrypted_list = []
        
        for i in range(len(list(list_of_encrypted))): #Per cadascun dels submissatges es crida a la funcio que els desencripta.
            decrypted_list.append(list(self.decryption(list_of_encrypted[i])))
        
        return decrypted_list #Es retorna una llista amb tots els submissatges desencriptats.
    
    
    
    def decrypt_file(self, e_filename, d_filename): #Donats dos fitxers, llegeix els coeficients dels polinomis corresponents al text encriptat i escriu el missatge desencriptat al segon fitxer.
                
        decrypted_lines_list = [] #Llista on els elements son les linies de text desencriptades.
        b_list = [] #Llista que conte subllistes corresponents als bits dels submissatges d'una linia de text desencriptada.
        b_def_list = [] #Llista que conte els bits dels submissatges unificats d'una linia de text desencriptada.
        pol_lines_list = [] #Llista que conte subllistes referents als coeficients dels missatges encriptats per cada linia de text.
        pol_lines_list = read_encrypted_file(e_filename, self.N) #Es llegeix el text encriptat del fitxer corresponent.

        for line in pol_lines_list: #Es desencripten les linies del fitxer una a una.
            b_list = self.decrypt_message(line) #Es desencripta una linia.
            add_zeros(b_list, self.N) #S'afegeixen els 0 que calgui al final de cada polinomi.
            b_def_list = [bit for polynomial in b_list for bit in polynomial] #S'unifiquen els coeficients dels polinomis a una sola linia.
            dec_text = "".join(map(chr, np.packbits(np.array(list(b_def_list))))) #Es transformen els bits al text en string corresponent.
            decrypted_lines_list.append(dec_text+"\n") #S'afegeix un salt de linia al final de cada linia.
        
        f = open(d_filename, "w") #S'escriu el text desencriptat al fitxer corresponent.
        f.writelines(decrypted_lines_list)
        f.close()
    
    
    
    def attack_file(self, e_filename, a_filename): #S'ataca el primer fitxer entrat per parametre i es guarda el missatge al segon fitxer entrat per parametre
        try:
            self.obtain_priv_keys(self.h) #S'obtenen les claus privades mitjançant l'atac.
            self.decrypt_file(e_filename, a_filename) #Es desencripta el fitxer.
    
        except ZeroDivisionError:
            print("Impossible obtenir les claus privades.")
    
    
    
    def obtain_priv_keys(self, h):
        
        self.h = h
        g_list =[0]*self.N #Llista que conte els coeficients de la clau privada g.
        one_list = [1]*(self.d+1) #Llista que conte els coeficients que valen 1 de la clau privada f.
        neg_one_list = [-1]*(self.d) #Llista que conte els coeficients que valen -1 de la clau privada f.
        zero_list = [0]*(self.N-self.d-1-self.d) #Llista que conte els coeficients que valen 0 de la clau privada f.
        self.f_list = one_list + neg_one_list + zero_list #Es concatenen el conjunt de 1, -1 i 0.
        z_pol_ring.<x> = PolynomialRing(ZZ)
        p_iter = sympy.utilities.iterables.multiset_permutations(self.f_list) #S'obtenen totes les possibles permutacions de la clau privada f.
       
        for p in itertools.takewhile(self.checking, p_iter):
            pass
            
        self.f = self.r_ring(self.f_list) #S'emmagatemen les claus privades f i g i els inversos modul p i q de f.
        self.g = self.r_ring(self.g_list)
        
        try:
            self.f_inv_mod_q = inv_pol_mod_power_2(self.r_ring_mod_q(self.f_list), self.r_ring_mod_2(self.f_list), self.N, self.q)
        
            if not self.p.is_power_of(2):
                self.f_inv_mod_p = inv_pol_mod_prime(self.r_ring_mod_p(self.f_list))
            
            else:
                self.f_inv_mod_p = inv_pol_mod_power_2(self.r_ring_mod_p(self.f_list), self.r_ring_mod_2(self.f_list), self.N, self.p)
        
        except ZeroDivisionError:
            raise
            
            
            
    def checking(self,p):
        
        z_pol_ring.<x> = PolynomialRing(ZZ)
        self.f_list = list(p)
        pol_f = z_pol_ring(self.f_list)
        pol_g = pol_f*self.h #Es calcula la convolucio.
        pol_g = center_lift(self.r_ring_mod_q(pol_g)) #Es prenen moduls p i (x^N-1) i es centre liften els coeficients.
        self.g_list = list(pol_g)

        if len(self.g_list) < self.N: #Si la llista dels seus coeficients es menor que N, s'afegeixen els 0 que calgui al final.  
            zeros = [0]*(self.N - len(self.g_list))
            self.g_list.extend(zeros)

        return not is_ternary(self.g_list, self.N, self.d)       

Tot seguit s'ataca el fitxer encriptat per la instància del criptosistema NTRU anterior mitjançant l'atac per força bruta i s'emmagatzema el missatge resultant a un fitxer de de text. A més a més, es computa el temps requerit per a atacar-lo.

In [11]:
m = Brute_force("public_key.txt")
start2_time = time.time()
m.attack_file("encriptacio_missatge.txt","atac_bf_missatge.txt")
print("L'atac per força bruta ha tardat %s segons." % (time.time() - start2_time))

L'atac per força bruta ha tardat 0.7700543403625488 segons.


A continuació s'ataca mitjançant l'atac per força bruta el missatge encriptat prèviament sense fer ús de fitxers de text. A més a més, es computa el temps requerit per a desencriptar-lo.

In [12]:
nfb = Brute_force(N, p, q, d)
nfstart4_time = time.time()
nfb.obtain_priv_keys(nfh)
nfbf = nfb.decryption_no_file(nfenc)
print("Missatge que es vol ecriptar: {}".format(nfmessage))
print("Missatge encriptat: {}".format(list(nfenc)))
print("Missatge desencriptat: {}".format(nfbf))
print("L'atac per força bruta ha tardat %s segons." % (time.time() - nfstart4_time))

Missatge que es vol ecriptar: [1, 1, 3, -1, -3, 2, 0, 3, -3, -1, 1, -3, 2, 0, 1, -3, -2]
Missatge encriptat: [137, -87, -77, -113, -83, 18, -160, 171, 253, 135, -127, 13, -214, -48, -151, -19, -162]
Missatge desencriptat: [1, 1, 3, -1, -3, 2, 0, 3, -3, -1, 1, -3, 2, 0, 1, -3, -2]
L'atac per força bruta ha tardat 0.7500321865081787 segons.


### Atac per trobada a mig camí

La classe Meet_in_the_middle es correspon amb la implementació de l'atac per trobada a mig camí. Permet desencriptar missatges a partir dels paràmetres públics i de la clau pública. A més a més, els textos encriptats i desencriptats es poden llegir i escriure en fitxers de text.

In [13]:
class Meet_in_the_middle:

    N = None
    p = None
    q = None
    d = None
    r = None
    h = None
    f = None
    f_inv_mod_p = None
    f_inv_mod_q = None
    g = None
    r_ring = None
    r_ring_mod_p = None
    r_ring_mod_q = None
    r_ring_mod_2 = None
    bins_a1= None
    bins_a2 = None
    trobat = False
    g_list = None
    f_list = None
    k = None

    

    def __init__(self, *args): #Inicialitzacio dels parametres i dels anells que es requereixen per fer funcionar l'atac al KRP basat en reticles sense llegir un fitxer.
        
        if len(args)==1:
            f = open(args[0], "r") #Es llegeix el fitxer que conte els parametres publics.
            params = f.read()
            f.close()

            param_list = [] #Llista que conte els parametres publics en format string.
            param_list  = params.split(";")
            param_list[-1] = param_list[-1].split(",") #Es separen els coeficients de la clau publica h llegida del fitxer.
            h_int_list = [int(c) for c in param_list[-1]] #Es passen de coeficients de tipus string a tipus int.

            z_pol_ring.<x> = PolynomialRing(ZZ)
            self.N = Integer(param_list[0]) #S'estableixen els nous valors dels parametres publics.
            self.p = Integer(param_list[1])
            self.q = Integer(param_list[2])
            self.d = Integer(param_list[3])
            self.h = z_pol_ring(h_int_list)
            
        else:
            try:
                assert (args[0] - 1)/2 >= args[3] #Comprovem la condició 1 dels parametres.

            except AssertionError: 
                print('Cal que (N - 1)/2 >= d')

            try:  
                assert args[2] > (6*args[3] + 1)*args[1] #Comprovem la condició 2 dels parametres.

            except AssertionError:
                print('Cal que q > (6*d + 1)*p')

            try:
                assert args[2].is_power_of(2) #Comprovem que q sigui potència de 2.

            except AssertionError:
                print('q ha de ser potència de 2.')

            self.N = args[0]
            self.p = args[1]
            self.q = args[2]
            self.d = args[3]
        
        aux_ring.<x> = PolynomialRing(ZZ)
        self.r_ring = aux_ring.quotient(x^self.N - 1)
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(self.p)) 
        self.r_ring_mod_p = aux_ring.quotient(x^self.N - 1) 
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(self.q)) 
        self.r_ring_mod_q = aux_ring.quotient(x^self.N - 1) 
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(2)) 
        self.r_ring_mod_2 = aux_ring.quotient(x^self.N - 1)



    def decryption(self, e): #Es desencripta el missatge entrat per parametre.

        z_pol_ring.<x> = PolynomialRing(ZZ)
        a = z_pol_ring(list(self.f))*e #Es calcula la primera de les dues convolucions.
        a = center_lift(self.r_ring_mod_q(a)) #Es prenen moduls q i (x^N-1) i es centre liften els coeficients.
        b = z_pol_ring(list(a))*self.f_inv_mod_p #Es calcula la segona de les dues convolucions.
        b = center_lift(self.r_ring_mod_p(b)) #Es prenen moduls p i (x^N-1) i es centre liften els coeficients.

        return b



    def decryption_no_file(self, e): #Desencripta missatges sense lectura de fitxers.
        
        b = self.decryption(e)
        b_list = list(b)
        
        if (len(b_list)) < self.N: #Si el tamany de la llista es menor que N, cal afegir els 0 restants al final.
            zeros = [0]*(self.N - len(b_list))
            b_list.extend(zeros)

        return b_list
    
    
    
    def decrypt_message(self, list_of_encrypted): #Es desencripten el conjunt de submissatges encriptats.

        decrypted_list = []

        for i in range(len(list(list_of_encrypted))): #Per cadascun dels submissatges es crida a la funcio que els desencripta.
            decrypted_list.append(list(self.decryption(list_of_encrypted[i])))

        return decrypted_list #Es retorna una llista amb tots els submissatges desencriptats.


    
    def decrypt_file(self, e_filename, d_filename): #Donats dos fitxers, llegeix els coeficients dels polinomis corresponents al text encriptat i escriu el missatge desencriptat al segon fitxer.

        decrypted_lines_list = [] #Llista on els elements son les linies de text desencriptades.
        b_list = [] #Llista que conte subllistes corresponents als bits dels submissatges d'una linia de text desencriptada.
        b_def_list = [] #Llista que conte els bits dels submissatges unificats d'una linia de text desencriptada.
        pol_lines_list = [] #Llista que conte subllistes referents als coeficients dels missatges encriptats per cada linia de text.
        pol_lines_list = read_encrypted_file(e_filename, self.N) #Es llegeix el text encriptat del fitxer corresponent.

        for line in pol_lines_list: #Es desencripten les linies del fitxer una a una.
            b_list = self.decrypt_message(line) #Es desencripta una linia.
            add_zeros(b_list, self.N) #S'afegeixen els 0 que calgui al final de cada polinomi.
            b_def_list = [bit for polynomial in b_list for bit in polynomial] #S'unifiquen els coeficients dels polinomis a una sola linia.
            dec_text = "".join(map(chr, np.packbits(np.array(list(b_def_list))))) #Es transformen els bits al text en string corresponent.
            decrypted_lines_list.append(dec_text+"\n") #S'afegeix un salt de linia al final de cada linia.

        f = open(d_filename, "w") #S'escriu el text desencriptat al fitxer corresponent.
        f.writelines(decrypted_lines_list)
        f.close()



    def attack_file(self, e_filename, a_filename): #S'ataca el primer fitxer entrat per parametre i es guarda el missatge al segon fitxer entrat per parametre.
        
        try:
            self.collision(self.h)
            self.obtain_priv_keys() #S'obtenen les claus privades mitjançant l'atac.
            self.decrypt_file(e_filename, a_filename) #Es desencripta el fitxer.
        
        except (ZeroDivisionError, TypeError):
            print("Impossible obtenir les claus privades.")
        
        except MemoryError:
            print("Espai de memòria insuficient: impossible obtenir les claus privades.")

            
            
    def collision(self, h): #S'obtenen les claus privades seguint l'atac per trobada a mig cami.
       
        self.h = h
        
        try:
            self.g_list =[0]*self.N #Llista que conte els coeficients de la clau privada g.
            a1_N = ceil(self.N/2) #Llista que conte els coeficients de la primera part de la clau privada f.
            a2_N = floor(self.N/2) #Llista que conte els coeficients de la segona part de la clau privada f.

            if self.d%2 == 0: #Repartiment dels 1 i -1 entre les parts del polinomi f quan d es parell.
                a1_one_list = [1]*Integer(self.d/2+1)
                a1_neg_one_list = [-1]*Integer(self.d/2)
                a1_zero_list = [0]*Integer(a1_N-self.d-1)
                a1_list = a1_one_list + a1_neg_one_list + a1_zero_list #Es concatenen el conjunt de 1, -1 i 0.
                a2_one_list = [1]*Integer(self.d/2)
                a2_neg_one_list = [-1]*Integer(self.d/2)
                a2_zero_list = [0]*Integer(a2_N-self.d)
                a2_list = a2_one_list + a2_neg_one_list + a2_zero_list #Es concatenen el conjunt de 1, -1 i 0.

            else: #Repartiment dels 1 i -1 entre les parts del polinomi f quan d es senar.
                a1_one_list = [1]*Integer(Integer(self.d+1)/2)
                a1_neg_one_list = [-1]*Integer(Integer(self.d+1)/2)
                a1_zero_list = [0]*Integer(a1_N-self.d-1)
                a1_list = a1_one_list + a1_neg_one_list + a1_zero_list #Es concatenen el conjunt de 1, -1 i 0.
                a2_one_list = [1]*Integer(Integer(self.d+1)/2)
                a2_neg_one_list = [-1]*Integer(Integer(self.d-1)/2)
                a2_zero_list = [0]*Integer(a2_N-self.d)
                a2_list = a2_one_list + a2_neg_one_list + a2_zero_list #Es concatenen el conjunt de 1, -1 i 0.

            z_pol_ring.<x> = PolynomialRing(ZZ)
            a1_iter = sympy.utilities.iterables.multiset_permutations(a1_list) #S'obtenen totes les possibles permutacions de la primera part de la clau privada f.
            a2_iter = sympy.utilities.iterables.multiset_permutations(a2_list) #S'obtenen totes les possibles permutacions de la segona part de la clau privada f.
            self.k = ceil(log((factorial(self.N)/(factorial(self.d/2+1)*factorial(self.d/2)*factorial(self.N-self.d-1))))) #Es calcula el nombre de calaixos que s'usaran per classificar.
            self.bins_a1 = [[] for Null in range(pow(2,self.k))] #Llista de tots els calaixos.
            self.bins_a2 = [[] for Null in range(pow(2,self.k))] #Llista de tots els calaixos.

            for a1_p in a1_iter: #Es classifiquen totes les possibles primeres parts del polinomi f
                addzeros_1 = [0]*(self.N-a1_N)
                pol_a1_list = addzeros_1 + list(a1_p)
                pol_a1 = z_pol_ring(pol_a1_list)
                pol_g1 = pol_a1*self.h  #Es calcula la convolucio.
                pol_g1 = self.r_ring_mod_q(pol_g1).lift().change_ring(z_pol_ring) #Es prenen moduls p i (x^N-1) i es centre liften els coeficients.
                g1_list = list(pol_g1)

                if len(g1_list) < self.N: #Si la llista dels seus coeficients es menor que N, s'afegeixen els 0 que calgui al final.
                    zeros = [0]*(self.N - len(g1_list))
                    g1_list.extend(zeros)

                g1_k_coef_list = g1_list[:self.k] #Es prenen els primers k coeficients del polinomi g1.
                self.classify(pol_a1_list, g1_k_coef_list, True) #Es classifica el polinomi g1.

            for a2_p in itertools.takewhile(self.checking, a2_iter): #Es classifiquen totes les possibles segones parts del polinomi f
                pass

        except MemoryError:
            raise
        
        
    
    def checking(self,a2_p):
        
        try:
            a2_N = floor(self.N/2)
            index = 0
            self.trobat = False
            z_pol_ring.<x> = PolynomialRing(ZZ)
            addzeros_2 = [0]*(self.N-a2_N)
            pol_a2_list = list(a2_p) + addzeros_2
            neg_pol_a2_list = [ x for x in pol_a2_list] #Es converteix el polinomi en negatiu.
            pol_a2 = z_pol_ring(neg_pol_a2_list)
            pol_g2 = -pol_a2*self.h #Es calcula la convolucio.
            pol_g2 = self.r_ring_mod_q(pol_g2).lift().change_ring(z_pol_ring) #Es prenen moduls p i (x^N-1) i es centre liften els coeficients.
            g2_list = list(pol_g2)
            
            if len(g2_list) < self.N: #Si la llista dels seus coeficients es menor que N, s'afegeixen els 0 que calgui al final.
                zeros = [0]*(self.N - len(g2_list))
                g2_list.extend(zeros)

            g2_k_coef_list = g2_list[:self.k] #Es prenen els primers k coeficients del polinomi g2.
            bins_list = []
            bins_list = self.classify(pol_a2_list, g2_k_coef_list, False) #Es classifica el polinomi g2.

            while not self.trobat and index <len(bins_list): #S'itera fins a trobar la parella de claus privades f i g.

                if len(self.bins_a1[bins_list[index]])>0: #Es comproven tots els calaixos que disposin de mes d'un polinomi.
                    i2=0
                    
                    while not self.trobat and i2 < len(self.bins_a1[bins_list[index]]):
                        pol_f_1 = z_pol_ring(self.bins_a1[bins_list[index]][i2])
                        pol_f_2 = z_pol_ring(pol_a2_list)
                        pol_f = pol_f_1+pol_f_2
                        self.f_list = list(pol_f)
                        pol_g = pol_f*self.h #Es calcula la convolucio.
                        pol_g = center_lift(self.r_ring_mod_q(pol_g)) #Es prenen moduls p i (x^N-1) i es centre liften els coeficients.
                        self.g_list = list(pol_g)

                        if len(self.g_list) < self.N: #Si la llista dels seus coeficients es menor que N, s'afegeixen els 0 que calgui al final.
                            zeros = [0]*(self.N - len(self.g_list))
                            self.g_list.extend(zeros)
                        
                        self.trobat = is_ternary(self.g_list, self.N, self.d) #Es comprova si el polinomi g compleix les condicions de la clau privada.
                        i2+=1
                        
                index+=1

            return not self.trobat
        
        except MemoryError:
            raise
    
    
    
    def obtain_priv_keys(self):
        
        z_pol_ring.<x> = PolynomialRing(ZZ)
       
        if self.trobat:
            try:
                self.f = self.r_ring(self.f_list) #S'emmagatemen les claus privades f i g i els inversos modul p i q de f.
                self.g = self.r_ring(self.g_list)
                self.f_inv_mod_q = inv_pol_mod_power_2(self.r_ring_mod_q(self.f_list), self.r_ring_mod_2(self.f_list), self.N, self.q)

                if not self.p.is_power_of(2):
                    self.f_inv_mod_p = inv_pol_mod_prime(self.r_ring_mod_p(self.f_list))

                else:
                    self.f_inv_mod_p = inv_pol_mod_power_2(self.r_ring_mod_p(self.f_list), self.r_ring_mod_2(self.f_list), self.N, self.p)
            
            except ZeroDivisionError:
                raise
            
            except MemoryError:
                raise



    def classify(self, pol_a1, g1_k_coef_list, b): #Es classifica el polinomi entrat als calaixos segons els bits dels seus primers k coeficients.
        
        try:
            first_bit_list = [] #Llista que conte els primers bits dels k primers coeficients.
            first_modified_bit_list = [] #Llista que conte els primers bits dels k primers coeficients modificats.
            number_1 = 0
            number_2 = 0
            num_of_bits = ceil(log(self.q,2)) #Es calcula el nombre de bits que tindra el valor resultant.
            first_bit_list, first_modified_bit_list, first_modified_bit_list_2 = self.convert_to_bits(g1_k_coef_list, num_of_bits) #Es calculen els primers bits de cada coeficient.
            list_of_first_bit_list = []
            list_of_first_bit_list.append(first_bit_list) #S'afegeix la concatenacio dels primers bits dels k primers coeficients.

            if not b: #En cas que estiguem classificant a2, cal modificar els k primers coeficients.
                
                for i in range(0,len(first_bit_list)): #Per cada bit de la llista, cal comprovar si al modificar el seu coeficients corresponent el valor del bit canvia.

                    if first_bit_list[i] != first_modified_bit_list[i]: #En cas que canvii al sumar 1, cal afegir totes les noves combinacions.
                        l = len(list_of_first_bit_list)

                        for e in range(0,l): #Iterem entre les combinacions existents fins ara per afegir les noves.
                            new_list = list_of_first_bit_list[e].copy()
                            new_list[i] = first_modified_bit_list[i]
                            list_of_first_bit_list.append(new_list)


                    if first_bit_list[i] != first_modified_bit_list_2[i]: #En cas que canvii al restar 1, cal afegir totes les noves combinacions.
                        l = len(list_of_first_bit_list)

                        for e in range(0,l): #Iterem entre les combinacions existents fins ara per afegir les noves.
                            new_list = list_of_first_bit_list[e].copy()
                            new_list[i] = first_modified_bit_list_2[i]
                            list_of_first_bit_list.append(new_list)

            counter = 0
            bins_list = []

            for list_of_bits in list_of_first_bit_list: #Per cada combinacio de bits resultant, cal obtenir el nombre corresponent.
                counter+=1
                
                for bit_1 in list_of_bits: #S'obte el nombre corresponent a concatenar els bits.
                    number_1 = (number_1 << 1) | bit_1
                    
                if b:
                    self.bins_a1[number_1].append(pol_a1) #S'afegeix el polinomi als calaixos.
                    
                else:
                    bins_list.append(number_1)
                    self.bins_a2[number_1].append(pol_a1) #S'afegeix el polinomi als calaixos.
                    
                number_1 = 0

            if not b: #En cas que estiguem classificant a2, cal retornar els indexs dels calaixos
                return bins_list
            
        except MemoryError:
            raise
        


    def convert_to_bits(self, g1_k_coef_list, num_of_bits): #Es calculen els primers bits de cada coeficient.
        
        bits = [] #Llista que conte l'equivalent en binari d'un coeficient donat.
        modified_bits = [] #Llista que conte l'equivalent en binari d'un coeficient modificat donat.
        modified_bits_2 = [] #Llista que conte l'equivalent en binari d'un coeficient modificat donat.
        first_bit_list = [] #Llista que conte tots el primer bit de cada coeficient.
        first_modified_bit_list = [] #Llista que conte tots el primer bit modificat de cada coeficient.
        first_modified_bit_list_2 = [] #Llista que conte tots el primer bit modificat de cada coeficient.
        
        for coef in g1_k_coef_list: #S'itera per tots els coeficients.
            coef_int=Integer(coef)
            bits = [int(x) for x in '{:020b}'.format(coef_int)] #Es tradueix a binari el coeficient en questio.
            bits = bits[(20-num_of_bits):] #S'eliminen els 0 de l'inici.
            first_bit_list.append(bits[0]) #S'afegeix a la llista el bit mes representatiu.
            modified_coef = (coef_int+1)%self.q #Es modifica el coeficient.
            modified_bits = [int(x) for x in '{:020b}'.format(modified_coef)]#Es tradueix a binari el coeficient en questio.
            modified_bits = modified_bits[(20-num_of_bits):] #S'eliminen els 0 de l'inici.
            first_modified_bit_list.append(modified_bits[0]) #S'afegeix a la llista el bit mes representatiu.
            modified_coef_2 = (coef_int-1)%self.q #Es modifica el coeficient.
            modified_bits_2 = [int(x) for x in '{:020b}'.format(modified_coef_2)]#Es tradueix a binari el coeficient en questio.
            modified_bits_2 = modified_bits_2[(20-num_of_bits):] #S'eliminen els 0 de l'inici.
            first_modified_bit_list_2.append(modified_bits_2[0]) #S'afegeix a la llista el bit mes representatiu.

        return first_bit_list, first_modified_bit_list, first_modified_bit_list_2


Tot seguit s'ataca el fitxer encriptat per la instància del criptosistema NTRU anterior mitjançant l'atac per trobada a mig camí i s'emmagatzema el missatge resultant a un fitxer de de text. A més a més, es computa el temps requerit per a atacar-lo.

In [14]:
quim = Meet_in_the_middle("public_key.txt")
start11_time = time.time()
quim.attack_file("encriptacio_missatge.txt","atac_mitm_missatge.txt")
print("L'atac per trobada a mig camí ha tardat %s segons." % (time.time() - start11_time))

L'atac per trobada a mig camí ha tardat 0.4156069755554199 segons.


A continuació s'ataca mitjançant l'atac per trobada a mig camí el missatge encriptat prèviament sense fer ús de fitxers de text. A més a més, es computa el temps requerit per a desencriptar-lo.

In [15]:
nfm = Meet_in_the_middle(N, p, q, d)
nfstart5_time = time.time()
nfm.collision(nfh)
nfm.obtain_priv_keys()
nfmitm = nfm.decryption_no_file(nfenc)
print("Missatge que es vol ecriptar: {}".format(nfmessage))
print("Missatge encriptat: {}".format(list(nfenc)))
print("Missatge desencriptat: {}".format(nfmitm))
print("L'atac per trobada a mig camí ha tardat %s segons." % (time.time() - nfstart5_time))

Missatge que es vol ecriptar: [1, 1, 3, -1, -3, 2, 0, 3, -3, -1, 1, -3, 2, 0, 1, -3, -2]
Missatge encriptat: [137, -87, -77, -113, -83, 18, -160, 171, 253, 135, -127, 13, -214, -48, -151, -19, -162]
Missatge desencriptat: [1, 1, 3, -1, -3, 2, 0, 3, -3, -1, 1, -3, 2, 0, 1, -3, -2]
L'atac per trobada a mig camí ha tardat 0.3146665096282959 segons.


### Atac al KRP mitjançant reticles

La classe Krp_lattice es correspon amb la implementació de l'atac al KRP mitjançant reticles. Permet desencriptar missatges a partir dels paràmetres públics i de la clau pública. A més a més, els textos encriptats i desencriptats es poden llegir i escriure en fitxers de text.

In [16]:
class Krp_lattice:
    
    N = None
    p = None
    q = None
    d = None
    r = None
    h = None
    f = None
    f_inv_mod_p = None
    f_inv_mod_q = None
    g = None
    r_ring = None
    r_ring_mod_p = None
    r_ring_mod_q = None
    r_ring_mod_2 = None
    
    
    def __init__(self, *args): #Inicialitzacio dels parametres i dels anells que es requereixen per fer funcionar l'atac al KRP basat en reticles sense llegir un fitxer.
        
        if len(args)==1:
            f = open(args[0], "r") #Es llegeix el fitxer que conte els parametres publics.
            params = f.read()
            f.close()

            param_list = [] #Llista que conte els parametres publics en format string.
            param_list  = params.split(";")
            param_list[-1] = param_list[-1].split(",") #Es separen els coeficients de la clau publica h llegida del fitxer.
            h_int_list = [int(c) for c in param_list[-1]] #Es passen de coeficients de tipus string a tipus int.

            z_pol_ring.<x> = PolynomialRing(ZZ)
            self.N = Integer(param_list[0]) #S'estableixen els nous valors dels parametres publics.
            self.p = Integer(param_list[1])
            self.q = Integer(param_list[2])
            self.d = Integer(param_list[3])
            self.h = z_pol_ring(h_int_list)
            
        else:
            try:
                assert (args[0] - 1)/2 >= args[3] #Comprovem la condició 1 dels parametres.

            except AssertionError: 
                print('Cal que (N - 1)/2 >= d')

            try:  
                assert args[2] > (6*args[3] + 1)*args[1] #Comprovem la condició 2 dels parametres.

            except AssertionError:
                print('Cal que q > (6*d + 1)*p')

            try:
                assert args[2].is_power_of(2) #Comprovem que q sigui potència de 2.

            except AssertionError:
                print('q ha de ser potència de 2.')

            self.N = args[0]
            self.p = args[1]
            self.q = args[2]
            self.d = args[3]
        
        aux_ring.<x> = PolynomialRing(ZZ)
        self.r_ring = aux_ring.quotient(x^self.N - 1)
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(self.p)) 
        self.r_ring_mod_p = aux_ring.quotient(x^self.N - 1) 
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(self.q)) 
        self.r_ring_mod_q = aux_ring.quotient(x^self.N - 1) 
        
        aux_ring.<x> = PolynomialRing(ZZ.quotient(2)) 
        self.r_ring_mod_2 = aux_ring.quotient(x^self.N - 1)
        
    
                
    def decryption(self, e): #Es desencripta el missatge entrat per parametre.
        
        z_pol_ring.<x> = PolynomialRing(ZZ)
        a = z_pol_ring(list(self.f))*e #Es calcula la primera de les dues convolucions.
        a = center_lift(self.r_ring_mod_q(a)) #Es prenen moduls q i (x^N-1) i es centre liften els coeficients.
        b = z_pol_ring(list(a))*self.f_inv_mod_p #Es calcula la segona de les dues convolucions.
        b = center_lift(self.r_ring_mod_p(b)) #Es prenen moduls p i (x^N-1) i es centre liften els coeficients.

        return b
    
    
    
    def decryption_no_file(self, e): #Desencripta missatges sense lectura de fitxers.
        
        b = self.decryption(e)
        b_list = list(b)
        
        if (len(b_list)) < self.N: #Si el tamany de la llista es menor que N, cal afegir els 0 restants al final.
            zeros = [0]*(self.N - len(b_list))
            b_list.extend(zeros)

        return b_list
    
    
    
    def decrypt_message(self, list_of_encrypted): #Es desencripten el conjunt de submissatges encriptats.
        
        decrypted_list = []
        
        for i in range(len(list(list_of_encrypted))): #Per cadascun dels submissatges es crida a la funcio que els desencripta.
            decrypted_list.append(list(self.decryption(list_of_encrypted[i])))
        
        return decrypted_list #Es retorna una llista amb tots els submissatges desencriptats.
    
    
    
    def decrypt_file(self, e_filename, d_filename): #Donats dos fitxers, llegeix els coeficients dels polinomis corresponents al text encriptat i escriu el missatge desencriptat al segon fitxer.
                
        decrypted_lines_list = [] #Llista on els elements son les linies de text desencriptades.
        b_list = [] #Llista que conte subllistes corresponents als bits dels submissatges d'una linia de text desencriptada.
        b_def_list = [] #Llista que conte els bits dels submissatges unificats d'una linia de text desencriptada.
        pol_lines_list = [] #Llista que conte subllistes referents als coeficients dels missatges encriptats per cada linia de text.
        pol_lines_list = read_encrypted_file(e_filename, self.N) #Es llegeix el text encriptat del fitxer corresponent.

        for line in pol_lines_list: #Es desencripten les linies del fitxer una a una.
            b_list = self.decrypt_message(line) #Es desencripta una linia.
            add_zeros(b_list, self.N) #S'afegeixen els 0 que calgui al final de cada polinomi.
            b_def_list = [bit for polynomial in b_list for bit in polynomial] #S'unifiquen els coeficients dels polinomis a una sola linia.
            dec_text = "".join(map(chr, np.packbits(np.array(list(b_def_list))))) #Es transformen els bits al text en string corresponent.
            decrypted_lines_list.append(dec_text+"\n") #S'afegeix un salt de linia al final de cada linia.
        
        f = open(d_filename, "w") #S'escriu el text desencriptat al fitxer corresponent.
        f.writelines(decrypted_lines_list)
        f.close()
    
    
    
    def attack_file(self, e_filename, a_filename): #S'ataca el primer fitxer entrat per parametre i es guarda el missatge
                                                   #al segon fitxer entrat per parametre.
        try:
            self.obtain_priv_keys(self.h) #S'obtenen les claus privades mitjançant l'atac.
            self.decrypt_file(e_filename, a_filename) #Es desencripta el fitxer.
        
        except ZeroDivisionError:
            print("Impossible obtenir les claus privades.")
    
    
    
    def obtain_priv_keys(self, h): #S'obtenen les claus privades seguint l'atac a KRP basat en reticles.
        
        self.h = h
        keys_pair_list = [] #Llista que conte els coeficients de les claus privades f i g.
        f_list = [] #Llista que conte els coeficients de la clau privada f.
        g_list = [] #Llista que conte els coeficients de la clau privada g.
        M = self.gen_M_matrix() #Es genera la matriu M que defineix el reticle.
        reduced_M = M.LLL(algrithm='NTL:LLL', fp= None) #S'aplica l'algorisme LLL per reduir la base del reticle.
        keys_pair_list = list(reduced_M[0]) #S'obtenen les claus privades f i g.
        f_list = keys_pair_list[:self.N] #Es separen les claus privades f i g.
        g_list = keys_pair_list[self.N:]
        self.f = self.r_ring(f_list) #S'emmagatemen les claus privades f i g i els inversos modul p i q de f.
        self.g = self.r_ring(g_list)
        
        try:
            self.f_inv_mod_q = inv_pol_mod_power_2(self.r_ring_mod_q(f_list), self.r_ring_mod_2(f_list), self.N, self.q)

            if not self.p.is_power_of(2):
                self.f_inv_mod_p = inv_pol_mod_prime(self.r_ring_mod_p(f_list))

            else:
                self.f_inv_mod_p = inv_pol_mod_power_2(self.r_ring_mod_p(f_list), self.r_ring_mod_2(f_list), self.N, self.p)
        
        except ZeroDivisionError:
        
            raise
       
    
    
    def gen_M_matrix(self): #Es genera la matriu M que defineix el reticle.
        
        Id = matrix.identity(self.N) #Es genera la submatriu identitat.
        q_matrix = self.q*Id #Es genera la submatriu q*identitat.
        zero_matrix = matrix(self.N) #Es genera la submatriu de zeros.
        h_matrix = self.gen_h_matrix() #Es genera la submatriu de la clau publica h.
        M = block_matrix(ZZ,[[Id,h_matrix],[zero_matrix,q_matrix]]) #S'ajunten les 4 matrius per construir la matriu M.
        
        return M
        
        
        
    def gen_h_matrix(self): #Es genera la submatriu de la clau publica h.
        
        h_rows_list = [] #Llista que conte totes les files de la submatriu.
        h_list = list(self.h)
        
        if len(h_list) < self.N: #Si la llista dels seus coeficients es menor que N, s'afegeixen els 0 que calgui al final.  
            zeros = [0]*(self.N - len(h_list))
            h_list.extend(zeros)
        
        h_rows_list.append(h_list) #S'afegeix la primera fila de la matriu, que correspon al polinomi h
        
        for i in range(1,self.N): #S'itera per la resta de files
        
            h_list = h_list[-1:] +  h_list[:-1] #Es roten en una posicio cap a la dreta els coeficients del polinomi h.
            h_rows_list.append(h_list) #S'afegeix la seguent fila de la matriu.
            
        h_matrix = matrix(ZZ, h_rows_list) ##Es genera la submatriu de la clau publica h a partir de les files creades.
        
        return h_matrix             

Tot seguit s'ataca el fitxer encriptat per la instància del criptosistema NTRU anterior mitjançant l'atac al KRP mitjançant reticles i s'emmagatzema el missatge resultant a un fitxer de de text. A més a més, es computa el temps requerit per a atacar-lo.

In [17]:
krp_ret = Krp_lattice("public_key.txt")
start3_time = time.time()
krp_ret.attack_file("encriptacio_missatge.txt","atac_krp_ret_missatge.txt")
print("L'atac al KRP mitjançant reticles ha tardat %s segons." % (time.time() - start3_time))

L'atac al KRP mitjançant reticles ha tardat 1.1574833393096924 segons.


A continuació s'ataca mitjançant l'atac al KRP mitjançant reticles el missatge encriptat prèviament sense fer ús de fitxers de text. A més a més, es computa el temps requerit per a desencriptar-lo.

In [18]:
nfk = Krp_lattice(N, p, q, d)
nfstart2_time = time.time()
nfk.obtain_priv_keys(nfh)
nfkrp = nfk.decryption_no_file(nfenc)
print("Missatge que es vol ecriptar: {}".format(nfmessage))
print("Missatge encriptat: {}".format(list(nfenc)))
print("Missatge desencriptat: {}".format(nfkrp))
print("L'atac al KRP mitjançant reticles ha tardat %s segons." % (time.time() - nfstart2_time))

Missatge que es vol ecriptar: [1, 1, 3, -1, -3, 2, 0, 3, -3, -1, 1, -3, 2, 0, 1, -3, -2]
Missatge encriptat: [137, -87, -77, -113, -83, 18, -160, 171, 253, 135, -127, 13, -214, -48, -151, -19, -162]
Missatge desencriptat: [1, 1, 3, -1, -3, 2, 0, 3, -3, -1, 1, -3, 2, 0, 1, -3, -2]
L'atac al KRP mitjançant reticles ha tardat 0.9006383419036865 segons.
